<a href="https://colab.research.google.com/github/rambasnet/DeepLearningMaliciousURLs/blob/master/Keras_Tensorflow_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Include needed files. 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import csv
import os
import sys
import glob
import operator
import time

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.utils.np_utils import to_categorical, normalize

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import TensorBoard

Using TensorFlow backend.


# Include Dataset


In [2]:
%%bash
URL=https://iscxdownloads.cs.unb.ca/iscxdownloads/ISCX-URL-2016/
FILES=(ISCXURL2016.zip) 
for FILE in ${FILES[*]}; do
    if [ ! -f "$FILE" ]; then
        printf "downloading %s\n" $FILE
        curl -O $URL$FILE
        # unzip files
        echo 'unzipping ' $FILE
        unzip -o $FILE #overwrite exiting files/folders if exists
    fi
done

### Check Dataset

In [3]:
! ls FinalDataset

All_BestFirst.csv	      Malware_Infogain.csv
All_BestFirst_test.csv	      Malware_Infogain_test.csv
All.csv			      Phishing_BestFirst.csv
All.csv.pickle		      Phishing.csv
All_Infogain.csv	      Phishing_Infogain.csv
All_Infogain_test.csv	      Phishing_Infogain_test.csv
Defacement_BestFirst.csv      Spam_BestFirst.csv
Defacement.csv		      Spam_BestFirst_test.csv
Defacement_Infogain.csv       Spam.csv
Defacement_Infogain_test.csv  Spam_Infogain.csv
Malware_BestFirst.csv	      Spam_Infogain_test.csv
Malware.csv		      URL


# Set some data
> Some data needs to be set, we need to ensure that constants are set properly. These are important but will not be used until later.

In [4]:
resultPath = 'results_keras_tensorflow'
if not os.path.exists(resultPath):
   print('result path {} created.'.format(resultPath))
   os.mkdir(resultPath)

In [5]:
dep_var = 'Label'
model_name="init"

In [6]:
cat_names = []
cont_names = []

## Analyze FinalDataset/All.csv file
> lets make sure that the files are properly added, this should look similar to the FASTAI experiments.

In [7]:
df = pd.read_csv('FinalDataset/All.csv', low_memory=False)

In [8]:
df.shape

(36707, 80)

# Show all dataset column names

In [9]:
df.columns

Index(['Querylength', 'domain_token_count', 'path_token_count',
       'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
       'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
       'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
       'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
       'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
       'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
       'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
       'ISIpAddressInDomainName', 'CharacterContinuityRate',
       'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
       'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
       'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
       'Directory_LetterCount', 'Filename_LetterCount',
       'Extension_LetterCount', 'Query_LetterCount', 'LongestPathToken

# Show the first rows of the dataset

In [10]:
df.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


# Show the last rows of the dataset

In [11]:
df.tail()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
36702,29,4,14,5.750000,12,3.666667,4,20,24,3,...,3,2,7,0.690555,0.791265,0.777498,0.690227,0.656684,0.796205,spam
36703,0,4,13,3.750000,8,8.461538,4,24,23,0,...,16,15,-1,0.665492,0.820010,0.879588,0.674400,0.674671,-1.000000,spam
36704,58,3,27,6.666666,16,3.375000,3,41,34,20,...,8,7,9,0.656807,0.801139,0.684777,0.713622,0.717187,0.705245,spam
36705,35,3,13,4.333334,9,3.600000,3,15,13,7,...,9,8,3,0.725963,0.897617,0.871049,0.745932,0.758824,0.790772,spam
36706,40,3,25,6.666666,16,3.250000,3,35,31,19,...,7,6,7,0.674351,0.801139,0.697282,0.730563,0.731481,0.769238,spam


# Functions for Testing
> Now that our data has been collected it is time to create functions that will be used in later tests.

In [12]:
def loadData(csvFile):
    pickleDump = '{}.pickle'.format(csvFile)
    if os.path.exists(pickleDump):
        df = pd.read_pickle(pickleDump)
    else:
        df = pd.read_csv(csvFile, low_memory=False)
        # clean data
        # strip the whitspaces from column names
        df = df.rename(str.strip, axis='columns')
        #df.drop(columns=[], inplace=True)
        # drop missing values/NaN etc.
        #df.dropna(inplace=True)
        # drop Infinity rows and NaN string from each column
        for col in df.columns:
            indexNames = df[df[col]=='Infinity'].index
            if not indexNames.empty:
                print('deleting {} rows with Infinity in column {}'.format(len(indexNames), col))
                df.drop(indexNames, inplace=True)
            indexNames = df[df[col]=='NaN'].index
            if not indexNames.empty:
                print('deleting {} rows with NaN in column {}'.format(len(indexNames), col))
                df.drop(indexNames, inplace=True)
        
        df.to_pickle(pickleDump)
    
    return df


In [13]:
def baseline_model(feature_layer,inputDim=-1):
    global model_name
    model = tf.keras.Sequential([
        feature_layer,
        Dense(128, activation='relu', input_shape=(inputDim,)),
    #print(f"out_shape[1]:{out_shape[1]}")
        Dense(32, activation='relu'),
        Dense(5, activation='softmax')
    ]) #This is the output layer

    print('Categorical Cross-Entropy Loss Function')
    model_name += "_categorical"
    model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
#         else:
#             model_name += "_binary"
#             print('Binary Cross-Entropy Loss Function')
#             model.compile(optimizer='adam',
#                     loss='binary_crossentropy',
#                     metrics=['accuracy'])
    return model

In [14]:
def encode_labels(dataframe):
    dataframe=dataframe.copy()
    data_y=dataframe.pop(dep_var)
    encoder = LabelEncoder()
    encoder.fit(data_y)
    data_y = encoder.transform(data_y)
    dummy_y = to_categorical(data_y)
    return dummy_y

In [15]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    
    dataframe=dataframe.copy()
    
    #Encode the labels as numeric values
    labels = encode_labels(dataframe)
    
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

# Test LoadData Function
> This will look just like the FastAI test, but we are using Tensor, so lets make sure it works.

In [16]:
df1 = loadData('FinalDataset/All.csv')

In [17]:
df1.columns


Index(['Querylength', 'domain_token_count', 'path_token_count',
       'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
       'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
       'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
       'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
       'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
       'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
       'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
       'ISIpAddressInDomainName', 'CharacterContinuityRate',
       'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
       'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
       'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
       'Directory_LetterCount', 'Filename_LetterCount',
       'Extension_LetterCount', 'Query_LetterCount', 'LongestPathToken

In [18]:
df1.shape

(36697, 80)

In [19]:
df1['NumberRate_Extension'][:10]

0    1.0
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    1.0
Name: NumberRate_Extension, dtype: float64

In [20]:
df1.shape

(36697, 80)

In [21]:
df1.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


  # Experimenting with Final Dataset/All.csv
  
  #### Total Samples for each Type

In [22]:
label = 'URL_Type_obf_Type'
lblTypes=set(df[label])
for lbl in lblTypes:
    print('| {} | {} |'.format(lbl, len(df[df[label] == lbl].index)))

| Defacement | 7930 |
| spam | 6698 |
| malware | 6712 |
| benign | 7781 |
| phishing | 7586 |


In [23]:
dataPath = 'FinalDataset'
dep_var = label
cont_names = list(set(df.columns) - set(cat_names) - set([dep_var]))

In [24]:
cont_names

['SymbolCount_Directoryname',
 'Entropy_URL',
 'pathDomainRatio',
 'URL_DigitCount',
 'argDomanRatio',
 'Directory_LetterCount',
 'sub-Directory_LongestWordLength',
 'isPortEighty',
 'domain_token_count',
 'domainlength',
 'Query_LetterCount',
 'charcompvowels',
 'CharacterContinuityRate',
 'Entropy_Afterpath',
 'Domain_LongestWordLength',
 'executable',
 'ISIpAddressInDomainName',
 'tld',
 'NumberRate_DirectoryName',
 'ArgLen',
 'ArgUrlRatio',
 'ldl_path',
 'host_letter_count',
 'argPathRatio',
 'dld_path',
 'Entropy_Domain',
 'Entropy_Extension',
 'delimeter_Domain',
 'dld_filename',
 'ldl_getArg',
 'spcharUrl',
 'URL_sensitiveWord',
 'NumberRate_FileName',
 'charcompace',
 'fileNameLen',
 'NumberRate_Domain',
 'avgdomaintokenlen',
 'NumberRate_URL',
 'URL_Letter_Count',
 'delimeter_Count',
 'LongestVariableValue',
 'pathurlRatio',
 'ldl_domain',
 'SymbolCount_Domain',
 'SymbolCount_Afterpath',
 'Arguments_LongestWordLength',
 'pathLength',
 'subDirLen',
 'SymbolCount_Extension',
 'N

# Cast column values to float

In [25]:
df1.argPathRatio = df1['argPathRatio'].astype('float')

# Experimenting with Tensorflow Keras

#### Globals for Testing


In [26]:
dataFile = 'All.csv'
optimizer='adam'
epochs=10
batch_size=10
feature_columns = []

#### Numeric Columns setup

In [27]:
#feature columns to classify malicious URLs
for header in ['dld_getArg',
 'URLQueries_variable',
 'Directory_DigitCount',
 'URL_sensitiveWord',
 'spcharUrl',
 'URL_Letter_Count',
 'SymbolCount_Directoryname',
 'host_DigitCount']:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [28]:
df1[dep_var]

0        Defacement
1        Defacement
2        Defacement
3        Defacement
4        Defacement
5        Defacement
6        Defacement
7        Defacement
8        Defacement
9        Defacement
10       Defacement
11       Defacement
12       Defacement
13       Defacement
14       Defacement
15       Defacement
16       Defacement
17       Defacement
18       Defacement
19       Defacement
20       Defacement
21       Defacement
22       Defacement
23       Defacement
24       Defacement
25       Defacement
26       Defacement
27       Defacement
28       Defacement
29       Defacement
            ...    
36677          spam
36678          spam
36679          spam
36680          spam
36681          spam
36682          spam
36683          spam
36684          spam
36685          spam
36686          spam
36687          spam
36688          spam
36689          spam
36690          spam
36691          spam
36692          spam
36693          spam
36694          spam
36695          spam


#### Training Setup


In [34]:
time_gen = int(time.time())
global model_name

batch_size = 32 # A small batch sized is used for demonstration purposes

model_name = f"{dataFile}_{time_gen}"

tensorboard = TensorBoard(log_dir='logs/{}'.format(model_name))

train_ds = df_to_dataset(df1, batch_size=batch_size)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
inputDim = 79
print('inputdim = ', inputDim)

encode_df = df1.copy()
encoded_y = encode_labels(encode_df)


y=df1[dep_var].astype('category')
X=df1.drop(dep_var,axis=1)

for index, (train_indices, val_indices) in enumerate(kfold.split(X, y)):
    xtrain, xval = X.iloc[train_indices], X.iloc[val_indices]
    ytrain, yval = y.iloc[train_indices], y.iloc[val_indices]
    
    model = baseline_model(feature_layer, inputDim)
    model.fit(x=xtrain.values, y=ytrain, epochs=epochs, validation_data=(xval,yval), callbacks=[tensorboard], batch_size=batch_size)

# train, test = train_test_split(df1, test_size=0.2)
# train, val = train_test_split(train, test_size=0.2)
# val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
# test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

inputdim =  79
[[0 4 5 ... nan -1.0 'Defacement']
 [0 4 5 ... 0.0 -1.0 'Defacement']
 [0 4 5 ... 0.0 -1.0 'Defacement']
 ...
 [58 3 27 ... 0.717186805 0.7052445629999999 'spam']
 [35 3 13 ... 0.7588236309999999 0.790771695 'spam']
 [40 3 25 ... 0.731481013 0.769237864 'spam']]
Categorical Cross-Entropy Loss Function


ValueError: in converted code:

    /usr/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column_v2.py:467 call *
        raise ValueError('We expected a dictionary here. Instead we got: ',

    ValueError: ('We expected a dictionary here. Instead we got: ', <tf.Tensor 'input_1_4:0' shape=(?, 79) dtype=float32>)


#### Create Data for Analysis

#### Define dimension of input values


#### Model Creation

In [ ]:
model.save('{}.model'.format(os.path.basename(dataPath)))

#### Setup Final Results

In [ ]:
scores = model.evaluate(test_ds, verbose=1)
print(model.metrics_names)
acc, loss=scores[1]*100, scores[0]*100
print('Baseline: accuracy: {:.2f}%: loss: {:.2f}'.format(acc, loss))

resultFile = os.path.join(resultPath, dataFile)
with open('{}.result'.format(resultFile), 'a') as fout:
  fout.write('{} results...'.format(model_name))
  fout.write('\taccuracy: {:.2f} loss: {:.2f}\n'.format(acc, loss))